### Custom mirge3 Library
mirge3 library include 2/6 nucleotides to the 5'/3' ends of each mature sequence. This script...
1. modifies that library by adding a 3rd nucleotide to the 5'-end. 
2. Ensures the added nucleotides match the assembly.
3. Ensures that the positions of the miRNA are reasonable (e.g. validates position against miRBase).
4. Uses gnomAD to get SNPs (e.g. does not use mirge3 SNPs).
5. Rebuilds index.Libs (e.g. ebwt) files. 
6. Rebuilds merges file (for SNPs).

**Before Running**:  
- Go here (https://www.ncbi.nlm.nih.gov/assembly/GCA_000001405.15) and download Refseq assembly.
- gunzip -d fna.gz
- samtools faidx GRCh38.p13_genomic.fna
- Create fasta file from index.lib  
`bowtie-inspect miRge3_Lib/human/index.Libs/human_mirna_miRBase > miRBase_annotations.fasta`

**Steps**: 
1. Grab annotations from mirge3 library (seperate SNPs).
2. Update annotations and check sequence against assembly.
3. Validate annotations against miRBase.

In [1]:
from pathlib import Path
import subprocess as SP
import re

samtools = Path(r"/storage/enc/vladlab/project_MDD/bin/samtools-1.10/samtools")
GRCh38 = Path(r"GCA_000001405.15_GRCh38_genomic.fna")

mirge3_anno = "/storage/enc/vladlab/project_MDD/scripts_main/23_0103_get_miRNA_annotations/miRBase_annotations.fasta"
miRBase = "/storage/enc/vladlab/project_MDD/bin/miRge3_Lib/human_23_0528/annotation.Libs/human_miRBase.gff3"
                             

chromosome_to_GenBank = {"1":"CM000663.2", 
                       "2":"CM000664.2",
                       "3":"CM000665.2",
                       "4":"CM000666.2",
                       "5":"CM000667.2",
                       "6":"CM000668.2",
                       "7":"CM000669.2",
                       "8":"CM000670.2",
                       "9":"CM000671.2",
                       "10":"CM000672.2",
                       "11":"CM000673.2",
                       "12":"CM000674.2",
                       "13":"CM000675.2",
                       "14":"CM000676.2",
                       "15":"CM000677.2",
                       "16":"CM000678.2",
                       "17":"CM000679.2",
                       "18":"CM000680.2",
                       "19":"CM000681.2",
                       "20":"CM000682.2",
                       "21":"CM000683.2",
                       "22":"CM000684.2",
                       "X":"CM000685.2", 
                       "Y":"CM000686.2",
                        "KI270908.1":"KI270908.1",
                        "KV766192.1":"KV766192.1"}


# Output
output = "miRBase_annotations.fasta"

### 1. Grab annotations from mirge3 library (seperate SNPs).

In [2]:
def grab_annotations_from_miRge3(fasta_file):
    anno_dict = {} # key: name,   value: (chromosome, start, stop, strand, sequence, 5'-end, 3'-end)
    
    with open(fasta_file, 'r') as f_in:
        to_record = False
        name = None
        chromsoome = None
        start = None
        stop = None
        strand = None
        org_sequence = None
        prune_sequence = None
        end5 = None
        end3 = None
        
        
        for line in f_in:
            line_split = line.split()
            if to_record and line.startswith(">"): 
                end5 = org_sequence[0:2]
                end3 = org_sequence[-6:]
                anno_dict[name] = [chromosome, start, stop, strand, prune_sequence, end5, end3]
                to_record = False
            
            if line.startswith(">"):
                to_record = True 
                name = line_split[0][1::]
                
                # Grab chromosome
                if "chr" in line_split[1]: # Note: Have unlocalized regions, like KI270908.1
                    chromosome = line_split[1][3::]
                else:
                    chromosome = line_split[1] 
                positions = line_split[3].split(":")
                strand = positions[1]
                start_stop = positions[2].split('-')
                if strand == "+":
                    start = int(start_stop[0]) + 2
                    stop = int(start_stop[1]) - 6
                elif strand == "-":
                    start = int(start_stop[0]) + 6
                    stop = int(start_stop[1]) - 2
                else:
                    print(f"ERROR!")
            else: 
                org_sequence = line.strip()
                prune_sequence = org_sequence[2:-6]
                
        # Record last entry
        if to_record:
            end5 = org_sequence[0:2]
            end3 = org_sequence[-6:]
            anno_dict[name] = [chromosome, start, stop, strand, prune_sequence, end5, end3]
        
                    
            
    return anno_dict

mirge3_anno_dict = {} # key: name,   value: (chromosome, start, stop, strand, sequence, end5, end3)
mirge3_anno_dict = grab_annotations_from_miRge3(mirge3_anno)

# Fixes 
mirge3_anno_dict["hsa-miR-3714-5p"][0] = "3"               # Position on KV766192.1, which isn't part of GCA_000001405.15
mirge3_anno_dict["hsa-miR-3714-5p"][1] = 16933195
mirge3_anno_dict["hsa-miR-3714-5p"][2] = 16933217

mirge3_anno_dict["hsa-miR-1231-5p.SNPA"][0] = "1"          # Other SNP correct
 
mirge3_anno_dict["hsa-miR-3198-3p.SNPC"][0] = "22"         # Other SNP correct
mirge3_anno_dict["hsa-miR-3198-3p.SNPC"][1] = 17764190
mirge3_anno_dict["hsa-miR-3198-3p.SNPC"][2] = 17764211
mirge3_anno_dict["hsa-miR-3198-3p.SNPC"][3] = "-"

mirge3_anno_dict["hsa-miR-1302-3p"][2] = 30458             # Stop position wrong

mirge3_anno_dict["hsa-miR-1303-3p"][0] = "5"               # All wrong
mirge3_anno_dict["hsa-miR-1303-3p"][1] =  154685826
mirge3_anno_dict["hsa-miR-1303-3p"][2] =  154685848
mirge3_anno_dict["hsa-miR-1303-3p"][3] = "+"  

mirge3_anno_dict["hsa-miR-1256-5p"][1] = 43237316          # Start position wrong

mirge3_anno_dict["hsa-miR-548u-3p"][0] = "6"               # All wrong
mirge3_anno_dict["hsa-miR-548u-3p"][1] =  57390179
mirge3_anno_dict["hsa-miR-548u-3p"][2] =  57390202 

mirge3_anno_dict["hsa-miR-378g"][1] =  26696355            # Start position wrong

mirge3_anno_dict["hsa-miR-4512-5p"][0] = "15"              # 3p is correct. 
mirge3_anno_dict["hsa-miR-4512-5p"][1] =  66497012 
mirge3_anno_dict["hsa-miR-4512-5p"][2] =  66497034
mirge3_anno_dict["hsa-miR-4512-5p"][3] =  "-"

# Removed because I don't trust it
del mirge3_anno_dict["hsa-miR-3921-3p"]                     # Indel based on assembly. Also extremely rare in our data. Perhaps miRNA sequence wrong?

# Removed after validating against miRBase
mirge3_anno_dict["hsa-miR-4801-5p"][0] = "4"               # Incorrect position/chromosome. 
mirge3_anno_dict["hsa-miR-4801-5p"][1] = 37241961
mirge3_anno_dict["hsa-miR-4801-5p"][2] = 37241984
mirge3_anno_dict["hsa-miR-4801-5p"][3] = "-"


mirge3_anno_dict["hsa-miR-663a-5p"][0] = "20"              # 5p position wrong according to miRBase        
mirge3_anno_dict["hsa-miR-663a-5p"][1] =  26208243 
mirge3_anno_dict["hsa-miR-663a-5p"][2] =  26208264
mirge3_anno_dict["hsa-miR-663a-5p"][3] =  "-"
mirge3_anno_dict["hsa-miR-663a-3p"][0] = "20"              # 3p doesn't exist according to miRBase. But sequence exist on stem-loop.
mirge3_anno_dict["hsa-miR-663a-3p"][1] = 26208211
mirge3_anno_dict["hsa-miR-663a-3p"][2] = 26208228
mirge3_anno_dict["hsa-miR-663a-3p"][3] = "-"


mirge3_anno_dict["hsa-miR-1256-5p"][0] =  "1"             # 5p position wrong according to miRBase.        
mirge3_anno_dict["hsa-miR-1256-5p"][1] =  20988377        # 5p chromosome different than 3p.
mirge3_anno_dict["hsa-miR-1256-5p"][2] =  20988398
mirge3_anno_dict["hsa-miR-1256-5p"][3] =  "-"
               

mirge3_anno_dict['hsa-miR-4315-5p'][0] =  "17"             # Wrong position
mirge3_anno_dict['hsa-miR-4315-5p'][1] = 45475408
mirge3_anno_dict['hsa-miR-4315-5p'][2] = 45475425
mirge3_anno_dict['hsa-miR-4315-5p'][3] = "-"
mirge3_anno_dict['hsa-miR-4315-3p'][0] = '17'              # 3p doesn't exist according to miRBase. But sequence exist on stem-loop
mirge3_anno_dict['hsa-miR-4315-3p'][1] = 45475377
mirge3_anno_dict['hsa-miR-4315-3p'][2] = 45475395
mirge3_anno_dict['hsa-miR-4315-3p'][3] = "-"


mirge3_anno_dict['hsa-miR-3648-5p'][1] = 8208500           # Positions for 5p and 3p off. 
mirge3_anno_dict['hsa-miR-3648-5p'][2] = 8208520 
mirge3_anno_dict['hsa-miR-3648-3p'][1] = 8208534
mirge3_anno_dict['hsa-miR-3648-3p'][2] = 8208553


mirge3_anno_dict['hsa-miR-4737-3p'][1] = 60043042          # Position for 3p slightly off (1000 bp)
mirge3_anno_dict['hsa-miR-4737-3p'][2] = 60043062
# 5p wasn't included for some reason. This is in miRBase database!
mirge3_anno_dict['hsa-miR-4737-5p'] = ("17", 60043077, 60043096, "-", 'ATGCGAGGATGCTGACAGTG', "GG", "CCTCAC")

mirge3_anno_dict['hsa-miR-1181-3p'][0] = '19'             # Position for 3p wrong. 
mirge3_anno_dict['hsa-miR-1181-3p'][1] = 10403468
mirge3_anno_dict['hsa-miR-1181-3p'][2] = 10403490
mirge3_anno_dict['hsa-miR-1181-3p'][3] = "-"

In [4]:
mirge3_anno_dict_no_SNPs = {} # key: name,   value: (chromosome, start, stop, strand, sequence, end5, end3)
mirge3_anno_dict_SNPs = {}    # key: name,   value: [sequence_1, sequence_2, ...]

for k, v in mirge3_anno_dict.items():
    name = k.split(".")[0]
    if ("SNP" in k) and ("SNPC" not in k):
        if name not in mirge3_anno_dict_SNPs:
            mirge3_anno_dict_SNPs[name] = [v[4]]
        else:
            mirge3_anno_dict_SNPs[name].append(v[4])
    else:
        mirge3_anno_dict_no_SNPs[name] = v
print(f"Number of entries in mirge3 library (excluding SNPs): {len(mirge3_anno_dict_no_SNPs)}")

Number of entries in mirge3 library (excluding SNPs): 2816


In [5]:
# Question: Where do SNPs occur in mirge3 library?
positions_of_SNPs = {} # key: name    value: [[position, reference_allele, SNP], [posiiton, reference_allele, SNP], ...]
in_isomiR_range = [] # isomiR define as SNPs 3-nucleotide deep on the 5' or 3' end. 

for k, SNPs in mirge3_anno_dict_SNPs.items():
    cannoical_seq = mirge3_anno_dict_no_SNPs[k][4]
    for SNP in SNPs:
        position = [i for i in range(len(cannoical_seq)) if cannoical_seq[i] != SNP[i]]
        if len(position) > 1:
            print(f"Error: Multiple SNP on same sequence!")
            
        reference_allele =  cannoical_seq[position[0]]
        SNP_allele = SNP[position[0]]
        
        if k in positions_of_SNPs:
            positions_of_SNPs[k].append([position[0], reference_allele, SNP_allele])
        else:
            positions_of_SNPs[k] = [[position[0], reference_allele, SNP_allele]]
        
        if position in [0,1,2, len(SNP), len(SNP)-1, len(SNP)-2]:
            in_isomiR_range.append(True)
        else:
            in_isomiR_range.append(False)

print(f"Number of miRNA with SNPs: {len(positions_of_SNPs)}")
print(f"Number of SNPs: {len(in_isomiR_range)}")
print(f"Number of SNPs that fall 3-nucleotides on the 5' or 3' end: {sum(in_isomiR_range)}")

Number of miRNA with SNPs: 238
Number of SNPs: 335
Number of SNPs that fall 3-nucleotides on the 5' or 3' end: 0


### 2. Update annotations and check sequence against assembly.

In [6]:
def do_query(query, strand):
    if strand == "+":
        result = SP.check_output([str(samtools), "faidx", str(GRCh38), query, "-n", "1000"])
    else:
        result = SP.check_output([str(samtools), "faidx", str(GRCh38), query, "-i", "-n", "1000"])
    result = result.decode('utf-8').split()
    new_sequence = result[1].upper()
    return new_sequence

In [7]:
mirge3_anno_dict_updated = {} # key: name,   value: (chromosome, new_start, stop, strand, prune_sequence, new_end5, new_end3)


stat_wrong_position = 0 
stat_wrong_position_fixed_by_StartPlusOne = 0
stat_perfect_match = 0
stat_inperfect_match = 0
stat_number_of_SNPS = 0

for miRNA, v in mirge3_anno_dict_no_SNPs.items():
    perfect_match = True # For tracking 5' and 3' matches
    chromosome, start, stop, strand, sequence, end5, end3 = v
    chromosome_number = chromosome_to_GenBank[chromosome]
    query = f"{chromosome_number}:{start}-{stop}"
    
    # Query Attempt #1
    new_sequence = do_query(query, strand)
    
    # Query Attempt #2: Adds +1 to the start (Common error)
    if sequence != new_sequence:
        stat_wrong_position += 1
        start = start + 1
        query = f"{chromosome_number}:{start}-{stop}"
        new_sequence = do_query(query, strand)
        if sequence == new_sequence:
            stat_wrong_position_fixed_by_StartPlusOne += 1
    
    # Final Check
    if sequence != new_sequence:
        print(f"  {miRNA} strand:{strand} chr:{chromosome} start:{start}  stop:{stop}")
        print(f"  Old sequence: {sequence}")
        print(f"  New sequence: {new_sequence}")
        
    # Get 5'-end and 3'-end
    if strand == "+":
        query = f"{chromosome_number}:{start-2}-{start-1}"
        new_end5 = do_query(query, strand)
        query = f"{chromosome_number}:{stop+1}-{stop+6}"
        new_end3 = do_query(query, strand)
    elif strand == "-":
        query = f"{chromosome_number}:{stop+1}-{stop+2}"
        new_end5 = do_query(query, strand)
        query = f"{chromosome_number}:{start-6}-{start-1}"
        new_end3 = do_query(query, strand)
        
    # Complete match?
    if (end5 != new_end5) or (end3 != new_end3):
        perfect_match = False
        
    if perfect_match:
        stat_perfect_match += 1
    else:
        stat_inperfect_match += 1
        
    # Record
    mirge3_anno_dict_updated[miRNA] = [chromosome, start, stop, strand, new_sequence, new_end5, new_end3]
        
print(f"Number of miRNA with perfect matches for 5' and 3' ends: {stat_perfect_match}")
print(f"Number of miRNA with imperfect matches for 5' and 3' ends: {stat_inperfect_match}")
print(f"Number of miRNA with wrong positions: {stat_wrong_position}")
print(f"Number of miRNA with wrong positions fixed by adding +1 to start: {stat_wrong_position_fixed_by_StartPlusOne}")
print(f"Number of SNPs added: {stat_number_of_SNPS}")

Number of miRNA with perfect matches for 5' and 3' ends: 2757
Number of miRNA with imperfect matches for 5' and 3' ends: 59
Number of miRNA with wrong positions: 232
Number of miRNA with wrong positions fixed by adding +1 to start: 232
Number of SNPs added: 0


### 3. Validate annotations against miRBase.  
- Note: A mature miRNA may align to multiple stem loops, which may come from multiple places on the genome.

In [8]:
miRBase_dict = {} # key: name,   value: [(chromosome,start, end), (chromosome,start, end), ...] 
pattern = "Name=(.*)"
with open(miRBase, 'r') as f_in:
    for line in f_in:
        if line.startswith("#"):
                continue
                
        line_split = line.split()
        chromosome = line_split[0][3::]
        miRNA_type = line_split[2]
        start = int(line_split[3])
        stop = int(line_split[4])
        if miRNA_type != "miRNA":
            continue

        name = re.search(pattern, line_split[8])
        name = name.group(1).split(";")[0] # In case a ; follows the Name
        
        if name in miRBase_dict:
            miRBase_dict[name].append((chromosome, start, stop))
        else:
            miRBase_dict[name] = [(chromosome, start, stop)]
            
print(f"Number of entries in miRBase: {len(miRBase_dict)}")

unique_to_mirge3 = set(mirge3_anno_dict_updated.keys()).difference(miRBase_dict.keys())
unique_to_miRBase = set(miRBase_dict.keys()).difference(mirge3_anno_dict_updated.keys())
in_common = set(mirge3_anno_dict_updated.keys()).intersection(miRBase_dict.keys())
print(f"Number of miRBase miRNAs unique to miRge3: {len(unique_to_mirge3)}")
print(f"Number of miRBase miRNAs unique to miRBase: {len(unique_to_miRBase)}")
print(f"Number of miRBase miRNAs in common: {len(in_common)}")

Number of entries in miRBase: 2652
Number of miRBase miRNAs unique to miRge3: 318
Number of miRBase miRNAs unique to miRBase: 154
Number of miRBase miRNAs in common: 2498


In [9]:
# Validated miRNA in common
stat_number_validated = 0 
for miRNA in in_common:
    validated = False
    
    for miRBase_entry in miRBase_dict[miRNA]:
        miRBase_chr, miRBase_start, miRBase_end = miRBase_entry
        mirge3_chr, mirge3_start, mirge3_end = mirge3_anno_dict_updated[miRNA][0:3]
        
        if (miRBase_chr == mirge3_chr) and (miRBase_start == mirge3_start) and (miRBase_end == mirge3_end):
            validated = True
        
    if validated:
        stat_number_validated += 1
    else:
        print(f"{miRNA} does not match!")
        print(f"miRBase entry: {miRBase_dict[miRNA]}")
        print(f"mirge3 entry: {mirge3_anno_dict_updated[miRNA]}")
        
print(f"Number of mirge3 miRNA validated in miRBase: {stat_number_validated}")

Number of mirge3 miRNA validated in miRBase: 2498


In [10]:
# Validated miRNA "unique" to miRge3
# Note: Many of these miRNAs likely had 5p/3p information appended OR they include the both arms 
#  where miRBase didn't. 

stat_number_strict_validated = 0 # Exact position
stat_number_rough_validated = 0  # Within 100 BP of each other
stat_number_not_validated = 0

for miRNA in unique_to_mirge3:
    # Try removing arm information
    new_name = "-".join(miRNA.split("-")[:-1])
    if new_name in miRBase_dict:
        strict_validated = False
        rough_validated = False
        
        for miRBase_entry in miRBase_dict[new_name]:
            miRBase_chr, miRBase_start, miRBase_end = miRBase_entry
            mirge3_chr, mirge3_start, mirge3_end = mirge3_anno_dict_updated[miRNA][0:3]
            
            if (miRBase_chr == mirge3_chr) and (miRBase_start == mirge3_start) and (miRBase_end == mirge3_end):
                strict_validated = True
            else: # rough validate check
                pos_difference = abs(miRBase_start - mirge3_start)
                if pos_difference <= 100:
                    rough_validated = True

        if strict_validated:
            stat_number_strict_validated += 1
        elif rough_validated:
            stat_number_rough_validated += 1
        else:
            print(f"{miRNA} does not match!")
            print(f"miRBase entry: {miRBase_dict[new_name]}")
            print(f"mirge3 entry: {mirge3_anno_dict_updated[miRNA]}")
            stat_number_not_validated += 1

print(f"Number of mirge3 miRNA, which were thought to be unique, strict validated in miRBase: {stat_number_strict_validated}")
print(f"Number of mirge3 miRNA, which were thought to be unique, rough validated in miRBase: {stat_number_rough_validated}")
print(f"Number of mirge3 miRNA, which were thought to be unique, not validated in miRBase: {stat_number_not_validated}")

Number of mirge3 miRNA, which were thought to be unique, strict validated in miRBase: 154
Number of mirge3 miRNA, which were thought to be unique, rough validated in miRBase: 153
Number of mirge3 miRNA, which were thought to be unique, not validated in miRBase: 0


In [11]:
# Recreate library file
with open(output, 'w') as f_out:
    for miRNA, v in mirge3_anno_dict_updated.items():
        chromosome, start, stop, strand, sequence, end5, end3 = v
        
        if strand == "+":
            start = start - 2
            stop = stop + 6
        else:
            start = start - 6
            stop = stop + 2
            
        new_sequence = end5+sequence+end3
        new_sequence_length = len(new_sequence)
        
        line1 = f">{miRNA} chr{chromosome} segs:1-{new_sequence_length} cds:{strand}:{start}-{stop}\n"
        line2 = new_sequence+"\n"
        f_out.write(line1)
        f_out.write(line2)